# Simulate an FMU with inputs
Tutorial (WIP) by Tobias Thummerer

## License

In [ ]:
# Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Code section

To run the example, the previously installed packages must be included. 

In [ ]:
# imports
using FMI
using FMIZoo
using Plots

### Simulation setup

Next, the start time and end time of the simulation are set. Finally, a step size is specified to store the results of the simulation at these time steps.

In [ ]:
tStart = 0.0
tStep = 0.01
tStop = 8.0
tSave = tStart:tStep:tStop

### Import FMU

In the next lines of code the FMU model from *FMIZoo.jl* is loaded and the information about the FMU is shown.

In [ ]:
# we use an FMU from the FMIZoo.jl
pathToFMU = get_model_filename("SpringPendulumExtForce1D", "Dymola", "2022x")

myFMU = fmiLoad(pathToFMU; type=:ME) # load FMU in ME-Mode ("Model Exchange")

fmiInfo(myFMU)

#### Simulate as Model-Exchange

In the function `fmiSimulateME()` the FMU is simulated in model-exchange mode (ME) with an adaptive step size but with fixed save points `tSave`. In addition, the start and end time are specified. Note, that the dynamics of the input variables are not considered by the steps ize control of the solver, so it is highly recommended to limit the solver step size with the keyword argument `dtmax` if the input is more dynamic than the system.

In [ ]:
# input function format "t", dependent on `t` (time)
function extForce_t(t)
    [sin(t)]
end 

# simulate while setting inputs
data_extForce_t = fmiSimulateME(myFMU, (tStart, tStop); saveat=tSave, inputValueReferences=["extForce"], inputFunction=extForce_t, dtmax=1e-2)
fmiPlot(data_extForce_t)

In [ ]:
# input function format "cxt", dependent on `c` (component), `x` (state) and `t` (time)
function extForce_cxt(c::Union{FMU2Component, Nothing}, x::Union{AbstractArray{fmi2Real}, Nothing}, t::fmi2Real)
    x1 = 0.0
    if x != nothing # this check is important, because inputs may be needed before the system state is known
        x1 = x[1] 
    end
    [sin(t) * x1]
end 

# simulate while setting inputs
data_extForce_cxt = fmiSimulateME(myFMU, (tStart, tStop); saveat=tSave, inputValueReferences=["extForce"], inputFunction=extForce_cxt, dtmax=1e-2)
fmiPlot(data_extForce_cxt)

### Unload FMU

After plotting the data, the FMU is unloaded and all unpacked data on disc is removed.

In [ ]:
fmiUnload(myFMU)